Nome:Thiago da Costa Gadelha<br>Matrícula:499284

# Recapitulação - Texto Introdutório

O objetivo deste projeto é fazer um modelo preditivo que consiga avaliar quem estará no pódio do prêmio de MVP(Neste projeto considerei os 4 primeiros, mas a NBA considera os 3 primeiros assim como qualquer premiação) e quem será o MVP.

Para isso, irei extrair as estatísticas da temporada regular de todos os jogadores da temporada 1979-80 até a temporada 2020-2021 para fazer o aprendizado do modelo de ML.Os dados de 2021-2022 serão a entrada para avaliar se o meu modelo consegue acertar os 4 primeiros lugares do prêmio de mvp deste ano.

Além de todas essas colunas que serão geradas na extração,no modelo de dataset proposto por mim, coloco as colunas 'mvp_candidate', 'mvp_podium' e 'mvp_winner'.Estas colunas são valores boleanos para avaliar essas condições.Também coloco a coluna 'mvp_votes_share'(coluna que armazena número de pontos recebidos dividido por número de pontos máximo.Caso o jogador não seja candidato a mvp, esse valor é 0).

As colunas descritas nos parágrafos acima tem como objetivo tentar montar um dataset em que seja possível usar a regressão linear para fazer a predição.Uso como valor de "chave" nomejogador_temporada_time para conseguir diferenciar apropiadamente os jogadores entre si e os valores numéricos são todos fornecidos pelos dados estatísticos.O objetivo é tentar prever o dado numérico de 'mvp_votes_share' de forma que a ordem seja a mesma do prêmio.

Obs:Tenho impressão de que vai haver mvp_votes_share negativos mesmo com uma filtragem, mas espero que ao menos dê para acertar o pódio

Neste notebook extrairei os dados do site basketball reference(https://www.basketball-reference.com/) para criar um dataset a partir de dados da  temporada regular da NBA.

# Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

***Como serão usados modelos de predição, o melhor parâmetro para avaliar a acuracia é por meio do r2score***

# Importando os dados

In [2]:
#Dados com os filtros aplicados no trabalho 2
dados = pd.read_csv('data/dataset_jogadores_melhorado.csv', sep = ',')
#Dados dos jogadores na temporada 2020-2021
dados_experimento = pd.read_csv('data/dataset_jogadores_2021.csv', sep = ';')

#Trocando o índice
dados = dados.set_index("player_season_team", drop=True)

#Trocando o índice
dados_experimento = dados_experimento.set_index("player_season_team", drop=True)
#Retirando os jogadores TOT
selecao = dados_experimento[dados_experimento.team == 'TOT'].index
dados_experimento.drop(selecao, inplace=True)

In [3]:
print(f"Quantidade de Jogadores:{dados.shape[0]}")
print(f"Quantidade de candidatos à MVP:{dados.query('mvp_candidate == True').shape[0]}")
print(f"Quantidade de MVPS:{dados.query('mvp_winner == True').shape[0]}")

Quantidade de Jogadores:8035
Quantidade de candidatos à MVP:670
Quantidade de MVPS:41


# Separando os registros de Treino e Teste

In [4]:
#Os dados de treino e teste foram setados manualmente, pois não consegui usar a correlação em conjunto com a temporada.
test_seasons = [2013, 2015, 1996, 2001, 1988, 2008, 2020]
dados_teste = dados[dados.season.isin(test_seasons)]
dados_treino = dados[~dados.season.isin(test_seasons)]
dados_treino.season.unique()

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1989, 1990, 1991,
       1992, 1993, 1994, 1995, 1997, 1998, 1999, 2000, 2002, 2003, 2004,
       2005, 2006, 2007, 2009, 2010, 2011, 2012, 2014, 2016, 2017, 2018,
       2019], dtype=int64)

# Separação das features

In [5]:
#Adicionando features que não são numéricas
nao_sao_features = ['player', 'season', 'pos', 'age', 'team', 'mvp_candidate', 'mvp_podium', 'mvp_winner','team_gb','mvp_votes_share']
colunas = list(dados.columns)
for remover in nao_sao_features:
    colunas.remove(remover)
colunas

['g',
 'gs',
 'mp',
 'fg',
 'fga',
 'fg_pct',
 'fg3',
 'fg3a',
 'fg3_pct',
 'fg2',
 'fg2a',
 'fg2_pct',
 'efg_pct',
 'ft',
 'fta',
 'ft_pct',
 'orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'mp_pergame',
 'fga_per_g',
 'fg3_per_g',
 'fg3a_per_g',
 'fg2_per_g',
 'fg2a_per_g',
 'ft_per_g',
 'fta_per_g',
 'orb_per_g',
 'drb_per_g',
 'trb_per_g',
 'ast_per_g',
 'stl_per_g',
 'blk_per_g',
 'tov_per_g',
 'pf_per_g',
 'pts_per_g',
 'fga_per_mp',
 'fg3_per_mp',
 'fg3a_per_mp',
 'fg2_per_mp',
 'fg2a_per_mp',
 'ft_per_mp',
 'fta_per_mp',
 'orb_per_mp',
 'drb_per_mp',
 'trb_per_mp',
 'ast_per_mp',
 'stl_per_mp',
 'blk_per_mp',
 'tov_per_mp',
 'pf_per_mp',
 'pts_per_mp',
 'fga_per_poss',
 'fg3_per_poss',
 'fg3a_per_poss',
 'fg2_per_poss',
 'fg2a_per_poss',
 'ft_per_poss',
 'fta_per_poss',
 'orb_per_poss',
 'drb_per_poss',
 'trb_per_poss',
 'ast_per_poss',
 'stl_per_poss',
 'blk_per_poss',
 'tov_per_poss',
 'pf_per_poss',
 'pts_per_poss',
 'off_rtg',
 'def_rtg',
 'fg3a_per_f

Como visto no trabalho anterior, há muitas features redudantes ou que não são relevantes para a análise, então deve-se reduzir o número de Features para testar o modelo da melhor forma.

In [6]:
#colunas_novo é referente a atributos que vi que seriam interessantes em um projeto do github
colunas_novo = ['obpm',
 'drb_per_g',
 'fga_per_poss',
 'fga_per_g',
 'tov_per_g',
 'fg_pct',
 'fga_per_poss',
 'dbpm',
 'pf_per_poss',
 'fg2a_per_mp',
 'pts_per_poss',
 'ows',
 'fg2a_per_g',
 'blk_per_g',
 'tov_pct',
 'fta_per_g',
 'trb_pct',
 'team_win_loss_pct',
 'ws',
 'def_rtg',
 'stl_per_g',
 'efg_pct',
 'tov_per_mp',
 'pf_per_g',
 'vorp',
 'ft_per_mp',
 'usg_pct',
 'drb_pct',
 'ast_per_g',
 'team_wins',
 'fta_per_poss',
 'fga_per_mp',
 'dws',
 'fga_per_g',
 'team_losses',
 'ws_per_48',
 'blk_per_mp',
 'g',
 'orb_per_g',
 'orb_per_mp',
 'fg3a_per_fga_pct',
 'blk_pct',
 'off_rtg',
 'pts_per_mp',
 'ft_pct',
 'ft_per_g',
 'fg2_per_mp',
 'fga_per_mp',
 'trb_per_g',
 'fg2_pct',
 'fta_per_fga_pct',
 'pts_per_g',
 'bpm',
 'mp',
 'ast_pct']

#colunas_sla é referente a atributos que foram selecionados no trabalho 2
colunas_sla = ['fg', 'fg2','pts', 'fg2_per_g', 'fga_per_g', 'pts_per_g', 'usg_pct', 'fta', 'drb', 'tov','fta_per_g', 'fta_per_mp','tov_per_g','ows', 'dws', 'ws', 'ws_per_48', 'obpm', 'bpm', 'vorp']

Classificação do prêmio de MVP da temporada 2020-2021<br>1° Nikola Jokic<br>2° Joel Embid<br>3° Stephen Curry<br>4° Giannis Antetokounmpo

# Regressão Linear

## Regressão Linear usando todas as colunas numéricas e todos os dados filtrados do trabalho 2

In [7]:
scaler = StandardScaler()

xTreino = dados_treino[colunas]
xTeste = dados_teste[colunas]
yTreino = dados_treino['mvp_votes_share']
yTeste = dados_teste['mvp_votes_share']
xExperimento = dados_experimento[colunas]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

model = LinearRegression()
model.fit(training_x_staggered, yTreino)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(yTeste, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 6594 elementos e testaremos com 1441 elementos
Acurácia:  39.49072052582891


In [8]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
GregWhittington_2021_DEN,Greg Whittington,2021,PF,27,DEN,4,0,12,0,3,...,25.0,0.653,5.0,115.1,110.1,4.82,False,False,False,0.660636
GaryClark_2021_DEN,Gary Clark,2021,SF,26,DEN,2,0,4,0,0,...,25.0,0.653,5.0,115.1,110.1,4.82,False,False,False,0.587230
AshtonHagans_2021_MIN,Ashton Hagans,2021,PG,21,MIN,2,0,4,0,0,...,49.0,0.319,29.0,112.1,117.7,-5.25,False,False,False,0.567078
IgnasBrazdeikis_2021_PHI,Ignas Brazdeikis,2021,SF,22,PHI,1,0,8,0,3,...,23.0,0.681,0.0,113.6,108.1,5.28,False,False,False,0.542922
JaylenAdams_2021_MIL,Jaylen Adams,2021,PG,24,MIL,7,0,18,1,8,...,26.0,0.639,3.0,120.1,114.2,5.57,False,False,False,0.435497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GaryPaytonII_2021_GSW,Gary Payton II,2021,PG,28,GSW,10,0,40,10,13,...,33.0,0.542,13.0,113.7,112.7,1.10,False,False,False,-0.358719
AnžejsPasečņiks_2021_WAS,Anžejs Pasečņiks,2021,C,25,WAS,1,0,6,0,1,...,38.0,0.472,15.0,116.6,118.5,-1.85,False,False,False,-0.377514
NorvelPelle_2021_NYK,Norvel Pelle,2021,C,27,NYK,9,0,52,5,7,...,31.0,0.569,8.0,107.0,104.7,2.13,False,False,False,-0.406052


## Regressão Linear usando os atributos do trabalho 2 e usando os dados filtrados do trabalho 2

In [9]:
scaler = StandardScaler()

xTreino = dados_treino[colunas_sla]
xTeste = dados_teste[colunas_sla]
yTreino = dados_treino['mvp_votes_share']
yTeste = dados_teste['mvp_votes_share']
xExperimento = dados_experimento[colunas_sla]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

model = LinearRegression()
model.fit(training_x_staggered, yTreino)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(yTeste, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 6594 elementos e testaremos com 1441 elementos
Acurácia:  33.33403524135781


In [10]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
AnžejsPasečņiks_2021_WAS,Anžejs Pasečņiks,2021,C,25,WAS,1,0,6,0,1,...,38.0,0.472,15.0,116.6,118.5,-1.85,False,False,False,1.487424
WillMagnay_2021_NOP,Will Magnay,2021,C,22,NOP,1,0,3,0,1,...,41.0,0.431,21.0,114.6,114.9,-0.20,False,False,False,1.216168
NoahVonleh_2021_BRK,Noah Vonleh,2021,PF,25,BRK,4,0,11,0,3,...,24.0,0.667,1.0,118.6,114.1,4.24,False,False,False,0.803262
ChrisSilva_2021_SAC,Chris Silva,2021,PF,24,SAC,4,0,9,1,3,...,41.0,0.431,21.0,113.7,117.4,-3.45,False,False,False,0.785747
JaylenAdams_2021_MIL,Jaylen Adams,2021,PG,24,MIL,7,0,18,1,8,...,26.0,0.639,3.0,120.1,114.2,5.57,False,False,False,0.661597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LouisKing_2021_SAC,Louis King,2021,SF,21,SAC,6,1,85,17,34,...,41.0,0.431,21.0,113.7,117.4,-3.45,False,False,False,-0.152834
MasonJones_2021_PHI,Mason Jones,2021,SG,22,PHI,6,0,27,5,9,...,23.0,0.681,0.0,113.6,108.1,5.28,False,False,False,-0.168500
GaryPaytonII_2021_GSW,Gary Payton II,2021,PG,28,GSW,10,0,40,10,13,...,33.0,0.542,13.0,113.7,112.7,1.10,False,False,False,-0.191958


## Regressão Linear usando os atributos do github e usando os dados filtrados do trabalho 2

In [11]:
scaler = StandardScaler()

xTreino = dados_treino[colunas_novo]
xTeste = dados_teste[colunas_novo]
yTreino = dados_treino['mvp_votes_share']
yTeste = dados_teste['mvp_votes_share']
xExperimento = dados_experimento[colunas_novo]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

model = LinearRegression()
model.fit(training_x_staggered, yTreino)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(yTeste, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 6594 elementos e testaremos com 1441 elementos
Acurácia:  39.6910372365597


In [12]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
GregWhittington_2021_DEN,Greg Whittington,2021,PF,27,DEN,4,0,12,0,3,...,25.0,0.653,5.0,115.1,110.1,4.82,False,False,False,0.859440
GaryClark_2021_DEN,Gary Clark,2021,SF,26,DEN,2,0,4,0,0,...,25.0,0.653,5.0,115.1,110.1,4.82,False,False,False,0.810019
IgnasBrazdeikis_2021_PHI,Ignas Brazdeikis,2021,SF,22,PHI,1,0,8,0,3,...,23.0,0.681,0.0,113.6,108.1,5.28,False,False,False,0.728994
TheoPinson_2021_NYK,Theo Pinson,2021,SG,25,NYK,17,0,34,1,9,...,31.0,0.569,8.0,107.0,104.7,2.13,False,False,False,0.695936
WillMagnay_2021_NOP,Will Magnay,2021,C,22,NOP,1,0,3,0,1,...,41.0,0.431,21.0,114.6,114.9,-0.20,False,False,False,0.606701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NorvelPelle_2021_NYK,Norvel Pelle,2021,C,27,NYK,9,0,52,5,7,...,31.0,0.569,8.0,107.0,104.7,2.13,False,False,False,-0.390957
GrantRiller_2021_CHO,Grant Riller,2021,PG,23,CHO,7,0,27,8,12,...,39.0,0.458,16.0,109.5,111.4,-1.94,False,False,False,-0.404152
GaryPaytonII_2021_GSW,Gary Payton II,2021,PG,28,GSW,10,0,40,10,13,...,33.0,0.542,13.0,113.7,112.7,1.10,False,False,False,-0.469834


## Regressão Linear usando todas as colunas numéricas e apenas os candidatos à MVP

In [13]:
scaler = StandardScaler()

xTreino = dados_treino.query('mvp_candidate == True')[colunas]
xTeste = dados_teste.query('mvp_candidate == True')[colunas]
yTreino = dados_treino.query('mvp_candidate == True')['mvp_votes_share']
yTeste = dados_teste.query('mvp_candidate == True')['mvp_votes_share']
xExperimento = dados_experimento[colunas]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

model = LinearRegression()
model.fit(training_x_staggered, yTreino)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(yTeste, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 563 elementos e testaremos com 107 elementos
Acurácia:  57.15537013220755


In [14]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
NorvelPelle_2021_SAC,Norvel Pelle,2021,C,27,SAC,1,0,4,0,1,...,41.0,0.431,21.0,113.7,117.4,-3.45,False,False,False,4.644106
ChrisSilva_2021_SAC,Chris Silva,2021,PF,24,SAC,4,0,9,1,3,...,41.0,0.431,21.0,113.7,117.4,-3.45,False,False,False,3.087085
KrisDunn_2021_ATL,Kris Dunn,2021,PG,26,ATL,4,0,45,1,12,...,31.0,0.569,8.0,113.7,111.4,2.14,False,False,False,2.531868
NoahVonleh_2021_BRK,Noah Vonleh,2021,PF,25,BRK,4,0,11,0,3,...,24.0,0.667,1.0,118.6,114.1,4.24,False,False,False,2.203177
WillMagnay_2021_NOP,Will Magnay,2021,C,22,NOP,1,0,3,0,1,...,41.0,0.431,21.0,114.6,114.9,-0.20,False,False,False,2.100020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NorvelPelle_2021_NYK,Norvel Pelle,2021,C,27,NYK,9,0,52,5,7,...,31.0,0.569,8.0,107.0,104.7,2.13,False,False,False,-1.549798
GrantRiller_2021_CHO,Grant Riller,2021,PG,23,CHO,7,0,27,8,12,...,39.0,0.458,16.0,109.5,111.4,-1.94,False,False,False,-2.273398
PatrickMcCaw_2021_TOR,Patrick McCaw,2021,SF,25,TOR,5,0,33,1,1,...,45.0,0.375,22.0,111.3,111.7,-0.54,False,False,False,-2.314536


## Regressão Linear usando os atributos do trabalho 2 e apenas os candidatos à MVP

In [15]:
scaler = StandardScaler()

xTreino = dados_treino.query('mvp_candidate == True')[colunas_sla]
xTeste = dados_teste.query('mvp_candidate == True')[colunas_sla]
yTreino = dados_treino.query('mvp_candidate == True')['mvp_votes_share']
yTeste = dados_teste.query('mvp_candidate == True')['mvp_votes_share']
xExperimento = dados_experimento[colunas_sla]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

model = LinearRegression()
model.fit(training_x_staggered, yTreino)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(yTeste, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 563 elementos e testaremos com 107 elementos
Acurácia:  51.377997759561175


In [16]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
NorvelPelle_2021_SAC,Norvel Pelle,2021,C,27,SAC,1,0,4,0,1,...,41.0,0.431,21.0,113.7,117.4,-3.45,False,False,False,2.101325
ElijahBryant_2021_MIL,Elijah Bryant,2021,SG,25,MIL,1,0,32,6,13,...,26.0,0.639,3.0,120.1,114.2,5.57,False,False,False,0.732074
VictorOladipo_2021_MIA,Victor Oladipo,2021,SG,28,MIA,4,4,111,16,43,...,32.0,0.556,9.0,108.1,108.0,-0.06,False,False,False,0.611104
RayjonTucker_2021_PHI,Rayjon Tucker,2021,PG,23,PHI,14,0,68,9,18,...,23.0,0.681,0.0,113.6,108.1,5.28,False,False,False,0.551394
AnžejsPasečņiks_2021_WAS,Anžejs Pasečņiks,2021,C,25,WAS,1,0,6,0,1,...,38.0,0.472,15.0,116.6,118.5,-1.85,False,False,False,0.531891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LouisKing_2021_SAC,Louis King,2021,SF,21,SAC,6,1,85,17,34,...,41.0,0.431,21.0,113.7,117.4,-3.45,False,False,False,-0.716290
RodionsKurucs_2021_BRK,Rodions Kurucs,2021,PF,22,BRK,5,0,16,1,3,...,24.0,0.667,1.0,118.6,114.1,4.24,False,False,False,-0.804253
GrantRiller_2021_CHO,Grant Riller,2021,PG,23,CHO,7,0,27,8,12,...,39.0,0.458,16.0,109.5,111.4,-1.94,False,False,False,-0.871130


## Regressão Linear usando os atributos do github e apenas os candidatos à MVP

In [17]:
scaler = StandardScaler()

xTreino = dados_treino.query('mvp_candidate == True')[colunas_novo]
xTeste = dados_teste.query('mvp_candidate == True')[colunas_novo]
yTreino = dados_treino.query('mvp_candidate == True')['mvp_votes_share']
yTeste = dados_teste.query('mvp_candidate == True')['mvp_votes_share']
xExperimento = dados_experimento[colunas_novo]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

model = LinearRegression()
model.fit(training_x_staggered, yTreino)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(yTeste, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 563 elementos e testaremos com 107 elementos
Acurácia:  56.47058357325122


In [18]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
GaryClark_2021_DEN,Gary Clark,2021,SF,26,DEN,2,0,4,0,0,...,25.0,0.653,5.0,115.1,110.1,4.82,False,False,False,2.963920
ChrisSilva_2021_SAC,Chris Silva,2021,PF,24,SAC,4,0,9,1,3,...,41.0,0.431,21.0,113.7,117.4,-3.45,False,False,False,2.582888
GregWhittington_2021_DEN,Greg Whittington,2021,PF,27,DEN,4,0,12,0,3,...,25.0,0.653,5.0,115.1,110.1,4.82,False,False,False,2.434879
TerranceFerguson_2021_PHI,Terrance Ferguson,2021,SG,22,PHI,13,0,49,1,7,...,23.0,0.681,0.0,113.6,108.1,5.28,False,False,False,2.388174
NoahVonleh_2021_BRK,Noah Vonleh,2021,PF,25,BRK,4,0,11,0,3,...,24.0,0.667,1.0,118.6,114.1,4.24,False,False,False,2.384387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DevonDotson_2021_CHI,Devon Dotson,2021,PG,21,CHI,11,0,50,11,21,...,41.0,0.431,18.0,110.7,111.6,-0.94,False,False,False,-1.250527
NorvelPelle_2021_NYK,Norvel Pelle,2021,C,27,NYK,9,0,52,5,7,...,31.0,0.569,8.0,107.0,104.7,2.13,False,False,False,-1.383001
GrantRiller_2021_CHO,Grant Riller,2021,PG,23,CHO,7,0,27,8,12,...,39.0,0.458,16.0,109.5,111.4,-1.94,False,False,False,-2.100145


# Árvore de Regressão

## Árvore de Regressão usando todos os atributos numéricos e os dados filtrados no trabalho 2

In [19]:
scaler = StandardScaler()
scaler_y = StandardScaler()
SEED = 42

xTreino = dados_treino[colunas]
xTeste = dados_teste[colunas]
yTreino = dados_treino['mvp_votes_share']
yTeste = dados_teste['mvp_votes_share']
xExperimento = dados_experimento[colunas]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

training_y_staggered = np.array(yTreino).reshape(-1,1)
test_y_staggered = np.array(yTeste).reshape(-1,1)

training_y_staggered = scaler_y.fit_transform(training_y_staggered)
test_y_staggered = scaler_y.transform(test_y_staggered)

model = DecisionTreeRegressor(max_depth=12, random_state=SEED)
model.fit(training_x_staggered, training_y_staggered)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(test_y_staggered, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 6594 elementos e testaremos com 1441 elementos
Acurácia:  44.47376000037831


In [20]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
JoelEmbiid_2021_PHI,Joel Embiid,2021,C,26,PHI,51,51,1585,461,899,...,23.0,0.681,0.0,113.6,108.1,5.28,True,True,False,5.510712
RussellWestbrook_2021_WAS,Russell Westbrook,2021,PG,32,WAS,65,65,2369,544,1238,...,38.0,0.472,15.0,116.6,118.5,-1.85,True,False,False,1.416983
NikolaJokić_2021_DEN,Nikola Jokić,2021,C,25,DEN,72,72,2488,732,1293,...,25.0,0.653,5.0,115.1,110.1,4.82,True,True,True,1.149495
NikolaVučević_2021_ORL,Nikola Vučević,2021,C,30,ORL,44,44,1500,436,908,...,51.0,0.292,28.0,104.0,113.3,-9.02,False,False,False,0.626149
DevinBooker_2021_PHO,Devin Booker,2021,SG,24,PHO,67,67,2270,623,1287,...,21.0,0.708,1.0,115.3,109.5,5.67,False,False,False,0.602889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JoshGreen_2021_DAL,Josh Green,2021,SG,20,DAL,39,5,445,42,93,...,30.0,0.583,10.0,112.4,110.2,2.26,False,False,False,-0.152309
IvicaZubac_2021_LAC,Ivica Zubac,2021,C,23,LAC,72,33,1609,257,394,...,25.0,0.653,5.0,114.0,107.8,6.02,False,False,False,-0.152309
LukaDončić_2021_DAL,Luka Dončić,2021,PG,21,DAL,66,66,2262,647,1351,...,30.0,0.583,10.0,112.4,110.2,2.26,True,False,False,-0.153055


## Árvore de Regressão usando os atributos do trabalho 2 e os dados filtrados no trabalho 2

In [21]:
scaler = StandardScaler()
scaler_y = StandardScaler()
SEED = 42

xTreino = dados_treino[colunas_sla]
xTeste = dados_teste[colunas_sla]
yTreino = dados_treino['mvp_votes_share']
yTeste = dados_teste['mvp_votes_share']
xExperimento = dados_experimento[colunas_sla]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

training_y_staggered = np.array(yTreino).reshape(-1,1)
test_y_staggered = np.array(yTeste).reshape(-1,1)

training_y_staggered = scaler_y.fit_transform(training_y_staggered)
test_y_staggered = scaler_y.transform(test_y_staggered)

model = DecisionTreeRegressor(max_depth=12, random_state=SEED)
model.fit(training_x_staggered, training_y_staggered)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(test_y_staggered, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 6594 elementos e testaremos com 1441 elementos
Acurácia:  41.041741087947194


In [22]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
JoelEmbiid_2021_PHI,Joel Embiid,2021,C,26,PHI,51,51,1585,461,899,...,23.0,0.681,0.0,113.6,108.1,5.28,True,True,False,7.999512
NikolaJokić_2021_DEN,Nikola Jokić,2021,C,25,DEN,72,72,2488,732,1293,...,25.0,0.653,5.0,115.1,110.1,4.82,True,True,True,3.882524
LukaDončić_2021_DAL,Luka Dončić,2021,PG,21,DAL,66,66,2262,647,1351,...,30.0,0.583,10.0,112.4,110.2,2.26,True,False,False,3.231249
MikeConley_2021_UTA,Mike Conley,2021,PG,33,UTA,51,51,1498,284,640,...,20.0,0.722,0.0,116.4,107.2,8.97,False,False,False,3.033541
GiannisAntetokounmpo_2021_MIL,Giannis Antetokounmpo,2021,PF,26,MIL,61,61,2013,626,1100,...,26.0,0.639,3.0,120.1,114.2,5.57,True,True,False,2.963761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JoshGreen_2021_DAL,Josh Green,2021,SG,20,DAL,39,5,445,42,93,...,30.0,0.583,10.0,112.4,110.2,2.26,False,False,False,-0.152628
BlakeGriffin_2021_DET,Blake Griffin,2021,PF,31,DET,20,20,626,81,222,...,52.0,0.278,29.0,106.6,111.1,-4.38,False,False,False,-0.152628
BlakeGriffin_2021_BRK,Blake Griffin,2021,PF,31,BRK,26,10,560,93,189,...,24.0,0.667,1.0,118.6,114.1,4.24,False,False,False,-0.152628


## Árvore de Regressão usando os atributos do github e os dados filtrados no trabalho 2

In [23]:
scaler = StandardScaler()
scaler_y = StandardScaler()
SEED = 42

xTreino = dados_treino[colunas_novo]
xTeste = dados_teste[colunas_novo]
yTreino = dados_treino['mvp_votes_share']
yTeste = dados_teste['mvp_votes_share']
xExperimento = dados_experimento[colunas_novo]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

training_y_staggered = np.array(yTreino).reshape(-1,1)
test_y_staggered = np.array(yTeste).reshape(-1,1)

training_y_staggered = scaler_y.fit_transform(training_y_staggered)
test_y_staggered = scaler_y.transform(test_y_staggered)

model = DecisionTreeRegressor(max_depth=12, random_state=SEED)
model.fit(training_x_staggered, training_y_staggered)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(test_y_staggered, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 6594 elementos e testaremos com 1441 elementos
Acurácia:  51.60161298346567


In [24]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
NikolaJokić_2021_DEN,Nikola Jokić,2021,C,25,DEN,72,72,2488,732,1293,...,25.0,0.653,5.0,115.1,110.1,4.82,True,True,True,6.976080
KevinDurant_2021_BRK,Kevin Durant,2021,PF,32,BRK,35,32,1157,324,603,...,24.0,0.667,1.0,118.6,114.1,4.24,False,False,False,5.510712
JamesHarden_2021_BRK,James Harden,2021,PG,31,BRK,36,35,1319,282,599,...,24.0,0.667,1.0,118.6,114.1,4.24,True,False,False,5.510712
JoelEmbiid_2021_PHI,Joel Embiid,2021,C,26,PHI,51,51,1585,461,899,...,23.0,0.681,0.0,113.6,108.1,5.28,True,True,False,2.963761
DonovanMitchell_2021_UTA,Donovan Mitchell,2021,PG,24,UTA,53,53,1771,478,1091,...,20.0,0.722,0.0,116.4,107.2,8.97,False,False,False,1.858920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JoshGreen_2021_DAL,Josh Green,2021,SG,20,DAL,39,5,445,42,93,...,30.0,0.583,10.0,112.4,110.2,2.26,False,False,False,-0.151765
IvicaZubac_2021_LAC,Ivica Zubac,2021,C,23,LAC,72,33,1609,257,394,...,25.0,0.653,5.0,114.0,107.8,6.02,False,False,False,-0.151765
GiannisAntetokounmpo_2021_MIL,Giannis Antetokounmpo,2021,PF,26,MIL,61,61,2013,626,1100,...,26.0,0.639,3.0,120.1,114.2,5.57,True,True,False,-0.153055


## Árvore de Regressão usando todos os atributos numéricos e apenas candidatos à MVP

In [25]:
scaler = StandardScaler()
scaler_y = StandardScaler()
SEED = 42

xTreino = dados_treino.query('mvp_candidate == True')[colunas]
xTeste = dados_teste.query('mvp_candidate == True')[colunas]
yTreino = dados_treino.query('mvp_candidate == True')['mvp_votes_share']
yTeste = dados_teste.query('mvp_candidate == True')['mvp_votes_share']
xExperimento = dados_experimento[colunas]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

training_y_staggered = np.array(yTreino).reshape(-1,1)
test_y_staggered = np.array(yTeste).reshape(-1,1)

training_y_staggered = scaler_y.fit_transform(training_y_staggered)
test_y_staggered = scaler_y.transform(test_y_staggered)

model = DecisionTreeRegressor(max_depth=12, random_state=SEED)
model.fit(training_x_staggered, training_y_staggered)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(test_y_staggered, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 563 elementos e testaremos com 107 elementos
Acurácia:  25.088116144794782


In [26]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
NikolaJokić_2021_DEN,Nikola Jokić,2021,C,25,DEN,72,72,2488,732,1293,...,25.0,0.653,5.0,115.1,110.1,4.82,True,True,True,3.054224
IvicaZubac_2021_LAC,Ivica Zubac,2021,C,23,LAC,72,33,1609,257,394,...,25.0,0.653,5.0,114.0,107.8,6.02,False,False,False,2.689066
RobertWilliams_2021_BOS,Robert Williams,2021,C,23,BOS,52,13,985,186,258,...,36.0,0.500,13.0,112.6,111.2,1.32,False,False,False,2.689066
PatrickMcCaw_2021_TOR,Patrick McCaw,2021,SF,25,TOR,5,0,33,1,1,...,45.0,0.375,22.0,111.3,111.7,-0.54,False,False,False,2.689066
TonyBradley_2021_PHI,Tony Bradley,2021,C,23,PHI,20,8,287,51,75,...,23.0,0.681,0.0,113.6,108.1,5.28,False,False,False,2.689066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CollinSexton_2021_CLE,Collin Sexton,2021,SG,22,CLE,60,60,2115,525,1105,...,50.0,0.306,27.0,103.8,112.3,-8.19,False,False,False,-0.595119
De'AaronFox_2021_SAC,De'Aaron Fox,2021,PG,23,SAC,58,58,2036,529,1110,...,41.0,0.431,21.0,113.7,117.4,-3.45,False,False,False,-0.595119
JaysonTatum_2021_BOS,Jayson Tatum,2021,SF,22,BOS,64,64,2290,605,1318,...,36.0,0.500,13.0,112.6,111.2,1.32,False,False,False,-0.595119


## Árvore de Regressão usando os atributos do trabalho 2 e apenas candidatos à MVP

In [27]:
scaler = StandardScaler()
scaler_y = StandardScaler()
SEED = 42

xTreino = dados_treino.query('mvp_candidate == True')[colunas_sla]
xTeste = dados_teste.query('mvp_candidate == True')[colunas_sla]
yTreino = dados_treino.query('mvp_candidate == True')['mvp_votes_share']
yTeste = dados_teste.query('mvp_candidate == True')['mvp_votes_share']
xExperimento = dados_experimento[colunas_sla]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

training_y_staggered = np.array(yTreino).reshape(-1,1)
test_y_staggered = np.array(yTeste).reshape(-1,1)

training_y_staggered = scaler_y.fit_transform(training_y_staggered)
test_y_staggered = scaler_y.transform(test_y_staggered)

model = DecisionTreeRegressor(max_depth=12, random_state=SEED)
model.fit(training_x_staggered, training_y_staggered)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(test_y_staggered, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 563 elementos e testaremos com 107 elementos
Acurácia:  30.75089178626136


In [28]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
NikolaJokić_2021_DEN,Nikola Jokić,2021,C,25,DEN,72,72,2488,732,1293,...,25.0,0.653,5.0,115.1,110.1,4.82,True,True,True,2.881461
GaryPaytonII_2021_GSW,Gary Payton II,2021,PG,28,GSW,10,0,40,10,13,...,33.0,0.542,13.0,113.7,112.7,1.10,False,False,False,2.689066
RobertWilliams_2021_BOS,Robert Williams,2021,C,23,BOS,52,13,985,186,258,...,36.0,0.500,13.0,112.6,111.2,1.32,False,False,False,2.689066
TonyBradley_2021_PHI,Tony Bradley,2021,C,23,PHI,20,8,287,51,75,...,23.0,0.681,0.0,113.6,108.1,5.28,False,False,False,2.689066
DerrickFavors_2021_UTA,Derrick Favors,2021,C,29,UTA,68,0,1039,155,243,...,20.0,0.722,0.0,116.4,107.2,8.97,False,False,False,2.689066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RuiHachimura_2021_WAS,Rui Hachimura,2021,PF,22,WAS,57,57,1797,310,648,...,38.0,0.472,15.0,116.6,118.5,-1.85,False,False,False,-0.597967
KyleGuy_2021_SAC,Kyle Guy,2021,PG,23,SAC,31,0,235,29,88,...,41.0,0.431,21.0,113.7,117.4,-3.45,False,False,False,-0.597967
BlakeGriffin_2021_BRK,Blake Griffin,2021,PF,31,BRK,26,10,560,93,189,...,24.0,0.667,1.0,118.6,114.1,4.24,False,False,False,-0.597967


## Árvore de Regressão usando os atributos do github e apenas candidatos à MVP

In [29]:
scaler = StandardScaler()
scaler_y = StandardScaler()
SEED = 42

xTreino = dados_treino.query('mvp_candidate == True')[colunas_novo]
xTeste = dados_teste.query('mvp_candidate == True')[colunas_novo]
yTreino = dados_treino.query('mvp_candidate == True')['mvp_votes_share']
yTeste = dados_teste.query('mvp_candidate == True')['mvp_votes_share']
xExperimento = dados_experimento[colunas_novo]

print(f'Treinaremos com {len(xTreino)} elementos e testaremos com {len(xTeste)} elementos')

training_x_staggered = scaler.fit_transform(xTreino)
test_x_staggered = scaler.transform(xTeste)
experimento_x = scaler.transform(xExperimento)

training_y_staggered = np.array(yTreino).reshape(-1,1)
test_y_staggered = np.array(yTeste).reshape(-1,1)

training_y_staggered = scaler_y.fit_transform(training_y_staggered)
test_y_staggered = scaler_y.transform(test_y_staggered)

model = DecisionTreeRegressor(max_depth=12, random_state=SEED)
model.fit(training_x_staggered, training_y_staggered)
predictions = model.predict(test_x_staggered)
predictions_experimento = model.predict(experimento_x)
r2 = r2_score(test_y_staggered, predictions) * 100

print('Acurácia: ', r2)

Treinaremos com 563 elementos e testaremos com 107 elementos
Acurácia:  18.044851966848753


In [30]:
dados_experimento['mvp_votes_share'] = predictions_experimento
dados_experimento.sort_values(['mvp_votes_share'], ascending = False)

,player,season,pos,age,team,g,gs,mp,fg,fga,...,team_losses,team_win_loss_pct,team_gb,team_pts_per_g,team_opp_pts_per_g,team_srs,mvp_candidate,mvp_podium,mvp_winner,mvp_votes_share
player_season_team,,,,,,,,,,,,,,,,,,,,,
NikolaJokić_2021_DEN,Nikola Jokić,2021,C,25,DEN,72,72,2488,732,1293,...,25.0,0.653,5.0,115.1,110.1,4.82,True,True,True,3.266252
IvicaZubac_2021_LAC,Ivica Zubac,2021,C,23,LAC,72,33,1609,257,394,...,25.0,0.653,5.0,114.0,107.8,6.02,False,False,False,2.689066
DewayneDedmon_2021_MIA,Dewayne Dedmon,2021,C,31,MIA,16,0,210,46,65,...,32.0,0.556,9.0,108.1,108.0,-0.06,False,False,False,2.689066
DamianJones_2021_LAL,Damian Jones,2021,C,25,LAL,8,6,112,16,17,...,30.0,0.583,10.0,109.5,106.8,2.77,False,False,False,2.689066
DanielGafford_2021_WAS,Daniel Gafford,2021,C,22,WAS,23,0,408,94,138,...,38.0,0.472,15.0,116.6,118.5,-1.85,False,False,False,2.689066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
QuinnCook_2021_LAL,Quinn Cook,2021,PG,27,LAL,16,0,62,12,26,...,30.0,0.583,10.0,109.5,106.8,2.77,False,False,False,-0.601289
Ty-ShonAlexander_2021_PHO,Ty-Shon Alexander,2021,SG,22,PHO,15,0,47,3,12,...,21.0,0.708,1.0,115.3,109.5,5.67,False,False,False,-0.601289
DariusBazley_2021_OKC,Darius Bazley,2021,PF,20,OKC,55,55,1714,273,690,...,50.0,0.306,30.0,105.0,115.6,-10.13,False,False,False,-0.601289
